In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np
import seaborn as sns

Objetivos do Script:

Esse script tem como objetivo ler os dados que foram capturados da nossa amostra piloto e realizar alguns estudos em cima desses dados. 

Os 6 primeiros tópicos que eu separei abaixo são utilizados na grande maioria dos scripts de tratamento de dados desse estudo. 
Deixei padronizado para que seja possivel tratar simulações com um dispositivo ou vários.

Sumário:
  - Leitura dos dados
  - Separar publicações e confirmação de entrega por dispositivo simulado.
  - Criação de um dataframe com informações capturadas. 
  - Medidas de tendência centrar por dispositivo
  - Medidas de tendência centrar de todos os dispositivos.
  - Gráfico de dispersão da amostra.
  - Calculo do tamanho da amostra.

Como estamos trabalhando com uma amostra piloto temos como objetivo calcular o tamanho das amostras isso será realizado na ultima parte desse script.


In [2]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data

,No.,Time,Source,Destination,Protocol,Length,Source Port,Message Identifier,Destination Port,Info
0,318,19.032647,192.168.15.8,130.185.238.170,MQTT,89,55897,NaN,1883,Connect Command
1,320,19.112437,130.185.238.170,192.168.15.8,MQTT,70,1883,NaN,55897,Connect Ack
2,321,19.112473,192.168.15.8,130.185.238.170,MQTT,354,55897,"1,2",1883,"Subscribe Request (id=1) [devices-1], Publish ..."
3,323,19.192029,130.185.238.170,192.168.15.8,MQTT,71,1883,1,55897,Subscribe Ack (id=1)
4,328,19.271864,130.185.238.170,192.168.15.8,MQTT,340,1883,2,55897,"Publish Message [devices-1], Publish Ack (id=2)"
...,...,...,...,...,...,...,...,...,...,...
3312,2816099,1888.334877,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,58693,Ping Response
3313,2816114,1889.444105,192.168.15.8,130.185.238.170,MQTT,68,55567,NaN,1883,Ping Request
3314,2816115,1889.522973,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,55567,Ping Response
3315,2816128,1891.085421,192.168.15.8,130.185.238.170,MQTT,68,59943,NaN,1883,Ping Request


In [3]:
#  - Separar publicações e confirmação de entrega por dispositivo simulado.
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  data = data.loc[ ~(data["Message Identifier"] == "1,2")]
  # data["Message Identifier"].dropna(replace=True)
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [4]:
devicesNew = devices.copy()
for index in devices:
    if len(devices[index]["publish"]) <= 0:
        ports = ports.pop(index)



In [5]:
# - Criação de um dataframe com informações capturadas acima. 
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [6]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 169.2174383585823
(Device-1)Median Round Trip Time = 164.25562550003292
(Device-1)Standard deviation  Round Trip Time = 13.201068216074399
(Device-1)Min Round Trip Time = 157.20754000005854
(Device-1)Max Round Trip Time = 216.3231220000057
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 156.5066011758781
(Device-2)Median Round Trip Time = 152.28359499997168
(Device-2)Standard deviation  Round Trip Time = 10.734223072257764
(Device-2)Min Round Trip Time = 146.21590199999446
(Device-2)Max Round Trip Time = 217.44502499996088
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 160.44964718592814
(Device-3)Median Round Trip Time = 154.60901900002
(Device-3)Standard deviation  Round Trip Time = 11.889595189393052
(Device-3)Min Round Trip Time = 150.8755720000181
(Device-3)Max Round Trip Time = 233.72675799998888
------------------------------------------------------

# CÁLCULO DO TAMANHO DA AMOSTRA

In [7]:
nivel_de_significancia = 0.05

In [8]:
nivel_de_confiança = 1 - nivel_de_significancia
nivel_de_confiança

0.95

In [9]:
#Calculando o z com scipy
z = scipy.stats.norm.ppf(0.975)
z

1.959963984540054

Com nosso z calculado podemos seguir adiante

### $$z = 1.959963984540054 $$

Abaixo segue algumas formulas que iremos utilizar para calcular o numero de amostra



### Calculando o tamanho da nossa amostra

In [10]:
df_concatenado = pd.concat(data.values())
desvio_padrao_amostral = df_concatenado["RTT"].std()
media = df_concatenado["RTT"].mean()
e = 0.05 * media
n = (z * (desvio_padrao_amostral/e))**2
n

10.69934448837131

In [11]:
n = (z * (desvio_padrao_amostral/e))**2
n

10.69934448837131

In [12]:
desvio_padrao_amostral

13.513914492163389